In [ ]:
# Updated imports for a more robust test harness.
import numpy as np
import pandas as pd
import time
import warnings
import datetime
from sklearn.datasets import (
    load_iris,
    load_diabetes,
    fetch_california_housing,
    load_wine,
    load_digits,
    load_breast_cancer,
    fetch_openml,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, mean_absolute_error
from sklearn.cluster import DBSCAN, KMeans
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.linear_model import SGDRegressor
from scipy.linalg import det
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool, cpu_count
from sklearn.neighbors import NearestNeighbors

# Suppress some common warnings that can clutter the output.
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Epsilon-based selection methods ---

def select_oips(X, eps):
    """
    Online Input Point Selection (OIPS) based on RBF kernel similarity.
    This optimized version uses an incremental lookup table for efficiency.

    Args:
        X (np.ndarray): The dataset.
        eps (float): The epsilon threshold for similarity.

    Returns:
        list: Indices of the selected points.
    """
    n_samples = len(X)
    if n_samples == 0:
        return []

    selected_indices = []

    # This array acts as our "lookup table"
    # It stores the maximum similarity found so far for each point.
    max_sims = np.zeros(n_samples)

    # Start with a random point to initialize the process.
    start_idx = np.random.randint(n_samples)
    selected_indices.append(start_idx)

    # Pre-calculate similarities to the first selected point.
    all_sims = pairwise_kernels(X, X[start_idx].reshape(1, -1), metric='rbf').flatten()
    max_sims = all_sims.copy()

    while True:
        # Check if the maximum similarity to any selected point is below epsilon.
        # This is the stopping criterion.
        if np.max(max_sims[selected_indices]) >= eps:
            break

        # Find the next point to select: the one with the lowest max similarity.
        remaining_indices = np.setdiff1d(np.arange(n_samples), selected_indices)
        if len(remaining_indices) == 0:
            break

        next_idx = remaining_indices[np.argmin(max_sims[remaining_indices])]
        selected_indices.append(next_idx)

        # Update the lookup table:
        # Calculate new similarities from the newly selected point.
        new_sims = pairwise_kernels(X, X[next_idx].reshape(1, -1), metric='rbf').flatten()
        # Update the max similarity for each point.
        max_sims = np.maximum(max_sims, new_sims)

    return selected_indices

def select_dbscan(X, eps, min_samples=5):
    """
    Selects cluster medoids using DBSCAN clustering.

    Args:
        X (np.ndarray): The dataset.
        eps (float): The epsilon parameter for DBSCAN.
        min_samples (int): The minimum number of samples to form a cluster.

    Returns:
        list: Indices of the selected medoid points.
    """
    labels = DBSCAN(eps=eps, min_samples=min_samples).fit_predict(X)
    selected_indices = []
    for lbl in np.unique(labels):
        if lbl == -1:
            continue  # Skip noise points.
        pts_indices = np.where(labels == lbl)[0]
        pts = X[pts_indices]
        # Find the point in the cluster closest to the cluster's mean (the medoid).
        medoid_idx = pts_indices[np.argmin(np.linalg.norm(pts - pts.mean(axis=0), axis=1))]
        selected_indices.append(medoid_idx)
    return selected_indices

def select_farthest(X, eps):
    """
    Farthest point sampling (FPS) with an epsilon-based stopping criterion.

    Args:
        X (np.ndarray): The dataset.
        eps (float): The epsilon threshold for stopping.

    Returns:
        list: Indices of the selected points.
    """
    n_samples = len(X)
    if n_samples == 0:
        return []

    # Pick a random starting point.
    start_idx = np.random.randint(n_samples)
    selected_indices = [start_idx]
    # Keep track of the squared distance to the nearest selected point.
    min_dists_sq = np.linalg.norm(X - X[start_idx], axis=1)**2

    while True:
        # Find the point that is farthest from all currently selected points.
        farthest_idx = np.argmax(min_dists_sq)
        # Stop if the farthest point is within the epsilon threshold.
        if np.sqrt(min_dists_sq[farthest_idx]) < eps:
            break
        selected_indices.append(farthest_idx)
        # Update distances with the newly selected point.
        new_dists_sq = np.linalg.norm(X - X[farthest_idx], axis=1)**2
        min_dists_sq = np.minimum(min_dists_sq, new_dists_sq)
    return selected_indices


# --- Proposed epsilon-based method ---

def compute_residuals(X, y):
    """
    Computes absolute residuals from a linear regression model.

    Args:
        X (np.ndarray): The feature matrix.
        y (np.ndarray): The target vector.

    Returns:
        np.ndarray: The array of absolute residuals.
    """
    # Using SGDRegressor for efficiency on large datasets.
    model = SGDRegressor(random_state=0).fit(X, y)
    y_pred = model.predict(X)
    residuals = np.abs(y - y_pred)
    return residuals

def select_proposed(X, y, eps=0.5, max_iter=1000):
    """
    The proposed selection method based on residuals and clustering.

    Args:
        X (np.ndarray): The feature matrix.
        y (np.ndarray): The target vector.
        eps (float): The epsilon parameter for DBSCAN.
        max_iter (int): Maximum number of iterations for the while loop.

    Returns:
        list: Indices of the selected points.
    """
    # 1) Precompute residuals
    resid = compute_residuals(X, y)

    # 2) Determine global percentile threshold
    resid_thr = np.percentile(resid, 50)

    selected_indices = []
    alive = np.ones(len(X), dtype=bool)
    it = 0

    while np.any(alive) and it < max_iter:
        it += 1

        # 3) Mask of points valid this iteration
        # Select points below the residual threshold and not yet selected.
        valid_indices = np.where(alive & (resid <= resid_thr))[0]
        if valid_indices.size == 0:
            break

        # 4) Cluster only the valid points
        X_valid = X[valid_indices]
        try:
            # Set a fixed, robust min_samples value.
            labels = DBSCAN(eps=eps, min_samples=5).fit_predict(X_valid)
        except ValueError:
            break

        # 5) Pick one representative per non-noise cluster
        new_indices = []
        for lbl in np.unique(labels):
            if lbl == -1:  # Noise points are not selected.
                continue

            cluster_indices_in_valid = np.where(labels == lbl)[0]
            orig_indices_in_cluster = valid_indices[cluster_indices_in_valid]

            # Select the point closest to the cluster mean.
            cluster_points = X[orig_indices_in_cluster]
            medoid_idx = orig_indices_in_cluster[np.argmin(np.linalg.norm(cluster_points - cluster_points.mean(axis=0), axis=1))]
            new_indices.append(medoid_idx)

        selected_indices.extend(new_indices)

        # 6) Remove all clustered points for the next iteration
        alive[valid_indices] = False

    return selected_indices

# --- Fixed-budget selection methods ---

def select_kmeans(X, y, M, task):
    """
    Selects cluster centroids from KMeans clustering.

    Args:
        X (np.ndarray): The feature matrix.
        y (np.ndarray): The target vector (not used for selection).
        M (int): The number of points to select.
        task (str): The task type ('classification' or 'regression').

    Returns:
        list: Indices of the selected cluster centroids.
    """
    M = min(M, len(X))
    if M == 0: return []

    km = KMeans(n_clusters=M, random_state=0, n_init='auto').fit(X)

    # Find the representative point for each cluster (the one closest to the center).
    selected_indices = []
    for i in range(M):
        cluster_indices = np.where(km.labels_ == i)[0]
        if len(cluster_indices) == 0:
            continue
        center = km.cluster_centers_[i]
        distances = np.linalg.norm(X[cluster_indices] - center, axis=1)
        selected_indices.append(cluster_indices[np.argmin(distances)])
    return selected_indices

def select_random(X, y, M, task):
    """
    Randomly selects M points from the dataset.

    Args:
        X (np.ndarray): The feature matrix.
        y (np.ndarray): The target vector (not used for selection).
        M (int): The number of points to select.
        task (str): The task type (not used for selection).

    Returns:
        list: Indices of the selected points.
    """
    return list(np.random.choice(len(X), min(M, len(X)), replace=False))


# --- Helper functions for a more robust evaluation harness ---

def get_datasets():
    """
    Loads and returns a dictionary of various datasets.

    The Egyptian Used Car Pricing dataset requires specific preprocessing
    to handle categorical and numeric data correctly.
    """
    # Load the Egyptian Used Car Pricing dataset from a local file.
    try:
        df = pd.read_csv('Egypt-Used-Car-Price.csv')
    except Exception as e:
        print(f"Failed to load dataset from local file. Please ensure the path is correct. Error: {e}")
        return {}

    # Separate features and target
    y = df['Price']
    X = df.drop('Price', axis=1)

    # --- Feature Engineering for EgyptianCarPricing dataset ---
    # Add car age as a new feature.
    current_year = datetime.datetime.now().year
    X['Car_Age'] = current_year - X['Year']
    # Drop the original 'Year' column, as it's now represented by 'Car_Age'.
    X = X.drop('Year', axis=1)
    # Drop Engine_CC as it's not a common feature
    X = X.drop('Engine_CC', axis=1)

    # Convert categorical columns to numeric using one-hot encoding
    X = pd.get_dummies(X, drop_first=True)

    # Convert all columns to a numeric format if they are not already.
    # This handles any remaining mixed data types.
    X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

    return {
        'EgyptianCarPricing': ('regression', (X.values, y.values))
    }

def find_optimal_epsilon(X, k=5):
    """
    Finds a suitable epsilon for DBSCAN by calculating the average distance
    to the k-th nearest neighbor. This is a common, reproducible heuristic.

    Args:
        X (np.ndarray): The dataset.
        k (int): The number of neighbors to consider.

    Returns:
        float: A suitable epsilon value for DBSCAN.
    """
    if len(X) < k:
        return 0.1 # Fallback for very small datasets.

    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    distances, _ = neigh.kneighbors(X)

    # We'll use the median of the k-th distances as a simple, robust heuristic.
    # Multiply by a small factor to make epsilon more generous.
    median_kth_distance = np.median(distances[:, -1])

    return 1.5 * median_kth_distance


def evaluate_selection_method(args):
    """
    Evaluates a single selection method on a given dataset.
    This function is designed to be run by a multiprocessing worker.
    """
    name, fn, X_tr, y_tr, X_te, y_te, task, M_budget, eps_val, dataset_name = args
    print(f"Starting {name} on {dataset_name}...")

    start_time = time.time()

    # Select points based on the method and its parameters.
    selected_indices = []
    if name == 'Proposed':
        selected_indices = fn(X_tr, y_tr, eps=eps_val)
        M_used = len(selected_indices)
    elif name in ['DBSCAN_eps', 'OIPS', 'Farthest']:
        selected_indices = fn(X_tr, eps=eps_val)
        M_used = len(selected_indices)
    else:  # Fixed-budget methods
        selected_indices = fn(X_tr, y_tr, M=M_budget, task=task)
        M_used = len(selected_indices)

    selection_time = time.time() - start_time

    # Handle cases where no points are selected or a single class is present.
    if M_used == 0 or (task == 'classification' and len(np.unique(y_tr[selected_indices])) < 2):
        print(f"Finished {name} on {dataset_name} with 0 selected points or insufficient classes.")
        return {
            'Dataset': dataset_name, 'Method': name, 'Task': task, 'M_used': M_used, 'NumPoints': 0,
            'SelTime(s)': selection_time, 'TrainTime(s)': None, 'TestTime(s)': None,
            'Accuracy': None, 'R2': None, 'MSE': None,
            'MAE': None, 'RMSE': None,
            'budget': M_budget, 'epsilon': eps_val
        }

    # Use the selected points to train the model.
    Zx, Zy = X_tr[selected_indices], y_tr[selected_indices]

    start_train_time = time.time()
    if task == 'classification':
        model = GaussianProcessClassifier(RBF(), copy_X_train=False).fit(Zx, Zy)
    else:
        model = GaussianProcessRegressor(RBF(), copy_X_train=False).fit(Zx, Zy)
    train_time = time.time() - start_train_time

    # Evaluate the trained model.
    start_test_time = time.time()
    preds = model.predict(X_te)
    test_time = time.time() - start_test_time

    acc, r2, mse, mae, rmse = None, None, None, None, None
    if task == 'classification':
        acc = accuracy_score(y_te, preds)
    else:
        r2 = r2_score(y_te, preds)
        mse = mean_squared_error(y_te, preds)
        mae = mean_absolute_error(y_te, preds)
        rmse = np.sqrt(mse)
        tol = 0.1 * np.std(y_tr)
        acc = np.mean(np.abs(preds - y_te) < tol)

    print(f"Finished {name} on {dataset_name}.")

    return {
        'Dataset': dataset_name, 'Method': name, 'Task': task, 'M_used': M_used, 'NumPoints': M_used,
        'SelTime(s)': round(selection_time, 4), 'TrainTime(s)': round(train_time, 4), 'TestTime(s)': round(test_time, 4),
        'Accuracy': round(acc, 4) if acc is not None else None,
        'R2': round(r2, 4) if r2 is not None else None,
        'MSE': round(mse, 4) if mse is not None else None,
        'MAE': round(mae, 4) if mae is not None else None,
        'RMSE': round(rmse, 4) if rmse is not None else None,
        'budget': M_used, 'epsilon': eps_val
    }

if __name__ == "__main__":
    results = []
    datasets = get_datasets()

    method_functions = {
        'Proposed': select_proposed,
        'DBSCAN_eps': select_dbscan,
        'OIPS': select_oips,
        'Farthest': select_farthest,
        'KMeans': select_kmeans,
        'Random': select_random,
    }

    all_tasks = []

    for dataset_name, (task, (X, y)) in datasets.items():
        print(f"--- Preparing evaluation tasks for dataset: {dataset_name} (task: {task}) ---")

        # Data splitting and scaling for features.
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=0)
        x_scaler = StandardScaler()
        X_tr_scaled = x_scaler.fit_transform(X_tr)
        X_te_scaled = x_scaler.transform(X_te)

        # Dynamically find a suitable epsilon using a data-driven heuristic.
        optimal_eps = find_optimal_epsilon(X_tr_scaled)

        # Create a relevant range of epsilons around the optimal one.
        # This is a key change for robustness and a more thorough evaluation.
        eps_multipliers = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 5.0, 10.0]
        eps_values_to_test = [optimal_eps * m for m in eps_multipliers]

        # Use the optimal epsilon to get the budget for this dataset.
        for current_eps in eps_values_to_test:
            selected_indices_proposed = select_proposed(X_tr_scaled, y_tr, eps=current_eps)
            M_used = len(selected_indices_proposed)

            # Add all tasks for this dataset to the list.
            # This will now be run in a single pass using multiprocessing.
            for name in ['Proposed']:
                 # Farthest and OIPS do not require the y_tr, so we pass in None
                if name == 'Proposed':
                    selected_indices_proposed = select_proposed(X_tr_scaled, y_tr, eps=current_eps)
                    M_used = len(selected_indices_proposed)
                    all_tasks.append((name, method_functions[name], X_tr_scaled, y_tr, X_te_scaled, y_te, task, M_used, current_eps, dataset_name))
                else:
                    all_tasks.append((name, method_functions[name], X_tr_scaled, y_tr, X_te_scaled, y_te, task, None, current_eps, dataset_name))

            for name in ['KMeans', 'Random']:
                all_tasks.append((name, method_functions[name], X_tr_scaled, y_tr, X_te_scaled, y_te, task, M_used, None, dataset_name))

    # Use a process pool to run all tasks in parallel.
    print(f"\n--- Running {len(all_tasks)} tasks using {cpu_count()} processes... ---")
    with Pool(cpu_count()) as pool:
        results = pool.map(evaluate_selection_method, all_tasks)

    # Convert results to a pandas DataFrame and print as a markdown table.
    df = pd.DataFrame(results)
    print("\n\n--- Final Results ---")
    print(df.to_markdown(index=False))


--- Preparing evaluation tasks for dataset: EgyptianCarPricing (task: regression) ---

--- Running 30 tasks using 16 processes... ---
